<a href="https://colab.research.google.com/github/AndrewCrossman/Graduate_Research/blob/main/BNL_Scripts/Convert_tth_to_xy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Convert_tth_to_xy**

*This code converts .chi files located in a google drive directory into .xy files*

In [1]:
from google.colab import drive  #allows downloading and accessing files on drive
from google.colab import files  #allows downloading and accessing files on drive
import shutil                   #allows uploading files to drive
import glob                     #useful for collecting and organizing files
import csv
import numpy as np              #math library
import pandas as pd             #data organization library
import scipy as sp              #math library
from scipy import optimize      #math fitting library
import matplotlib.pylab as plt  #allows for plotting
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [2]:
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [3]:
!ls '/content/drive/Shareddrives/BNL_June_2024/DataFiles/CeO2_PDF_600_1sec/integration'

CeO2_PDF_600_1sec_20240612-112457_67ac61_primary-100_mean_q.chi
CeO2_PDF_600_1sec_20240612-112457_67ac61_primary-100_mean_tth_v1.xy
CeO2_PDF_600_1sec_20240612-112457_67ac61_primary-100_mean_tth.xy
CeO2_PDF_600_1sec_20240612-112457_67ac61_primary-101_mean_q.chi
CeO2_PDF_600_1sec_20240612-112457_67ac61_primary-101_mean_tth_v1.xy
CeO2_PDF_600_1sec_20240612-112457_67ac61_primary-101_mean_tth.xy
CeO2_PDF_600_1sec_20240612-112457_67ac61_primary-102_mean_q.chi
CeO2_PDF_600_1sec_20240612-112457_67ac61_primary-102_mean_tth_v1.xy
CeO2_PDF_600_1sec_20240612-112457_67ac61_primary-102_mean_tth.xy
CeO2_PDF_600_1sec_20240612-112457_67ac61_primary-103_mean_q.chi
CeO2_PDF_600_1sec_20240612-112457_67ac61_primary-103_mean_tth_v1.xy
CeO2_PDF_600_1sec_20240612-112457_67ac61_primary-103_mean_tth.xy
CeO2_PDF_600_1sec_20240612-112457_67ac61_primary-104_mean_q.chi
CeO2_PDF_600_1sec_20240612-112457_67ac61_primary-104_mean_tth_v1.xy
CeO2_PDF_600_1sec_20240612-112457_67ac61_primary-104_mean_tth.xy
CeO2_PDF_600_1s

In [5]:
directory_path = '/content/drive/Shareddrives/BNL_June_2024/DataFiles/CeO2_PDF_600_1sec/integration'
tth_files = glob.glob(directory_path + '/*tth.xy')

In [22]:
data = pd.read_csv(tth_files[0])
data

,0,7.968615353662056311e-03 2.631737709045410156e+01
0,1,2.390584606098617240e-02 2.811417198181152344e+01
1,2,3.984307676831028155e-02 2.888211822509765625e+01
2,3,5.578030747563440112e-02 2.992591094970703125e+01
3,4,7.171753818295850680e-02 2.981155586242675781e+01
4,5,8.765476889028261942e-02 2.962524414062500000e+01
...,...,...
2994,2995,4.773997458378938319e+01 1.989528656005859375e+02
2995,2996,4.775591181449670586e+01 1.521253204345703125e+02
2996,2997,4.777184904520402853e+01 1.062618179321289062e+02
2997,2998,4.778778627591135120e+01 6.125721359252929688e+01


In [25]:
for file in tth_files:
  xy_name = file.replace(".xy","_v1.xy")
  data = pd.read_csv(file)
  data.to_csv(xy_name, header=False, sep='\t', index=False)  #downloads csv file directly to drive with no header, tab separations, and no index
  #print(xy_name)

In [28]:
'''
'''
data = pd.read_csv('/content/drive/Shareddrives/BNL_June_2024/DataFiles/CeO2_PDF_600_1sec/integration/CeO2_PDF_600_1sec_20240612-112457_67ac61_primary-93_mean_tth.xy',header=None)

#data.drop(data.columns[[0]], axis=1, inplace=True)
#data[['a', 'b']] = data[data.columns[0]].str.split(' ', expand=True)
#data = data[data.columns[0]].str.split(' ', expand=True)
data = float(data)
data.to_csv('/content/drive/Shareddrives/BNL_June_2024/DataFiles/CeO2_PDF_600_1sec/integration/CeO2_PDF_600_1sec_20240612-112457_67ac61_primary-93_mean_tth_v1.xy', header=False, sep='\t', index=False)  #downloads csv file directly to drive with no header, tab separations, and no index

TypeError: float() argument must be a string or a real number, not 'DataFrame'